In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 接下來的代碼
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input,Dropout
from tensorflow.keras.models import Sequential, load_model
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.utils import to_categorical
from tensorflow.keras.models import load_model

# 忽略警告
import warnings
warnings.simplefilter("ignore")


# 屏蔽特定類型的警告

# 初始化圖像和標籤列表
images = []
labels = []

# 初始化驗證集的資料
val_images_accumulated = []
val_labels_accumulated = []

# 刪除檔案
# 設置模型保存路徑
model_path = 'saved_model.h5'
if os.path.exists(model_path):
    os.remove(model_path)
    print(f"檔案 '{model_path}' 已成功刪除。")
else:
    print(f"檔案 '{model_path}' 不存在，無需刪除。")

# 檢查是否已經存在保存的模型，如果是則加載它
if os.path.exists(model_path):
    model = load_model(model_path)
else:
    # 如果模型不存在，則創建一個新的模型

    model = Sequential([
      Input(shape=(64, 64,1)),
      # Reshape to add a single channel dimension
      Conv2D(128, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Conv2D(128, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Conv2D(128, (3, 3), activation='relu'),
      Flatten(),
      Dense(256, activation='relu'),
      Dense(10, activation='softmax')
    ])


# 讀取標籤資料
for filename in os.listdir('/kaggle/input/advancedml-hw2/hw2/HW2_MNIST_train'):

    if filename.endswith('.txt'):
        labels_file_path = os.path.join('/kaggle/input/advancedml-hw2/hw2/HW2_MNIST_train', filename)
        with open(labels_file_path, 'r') as file:
            lines = file.readlines()

        # 處理每張圖像
        for line in lines:
            parts = line.strip().split('\t')
            digit = int(parts[0])
            image_path = os.path.join('/kaggle/input/advancedml-hw2/hw2/HW2_MNIST_train', filename.replace('.txt', '.png'))
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (64, 64))
            if img is None:
                print(f"Error reading image: {image_path}")
                continue
            digit_img = img.astype(np.float32) / 255.0
            images.append(digit_img)
            labels.append(digit)
            #print('label=',labels)
            # 如果已經收集到1000張圖片，則執行訓練
            if len(images) == 1000:
                # 將列表轉換為NumPy數組
                images_np = np.array(images)
                labels_np = np.array(labels)

                # 將標籤進行獨熱編碼
                labels_one_hot = to_categorical(labels_np, num_classes=10)

                # 加載模型
                if os.path.exists(model_path):
                    model = load_model(model_path)

                # 將數據集劃分為訓練集和驗證集7:3
                train_images, val_images, train_labels, val_labels = train_test_split(images_np, labels_one_hot, test_size=0.3, random_state=42)

                # 累積驗證集的資料
                val_images_accumulated.extend(val_images)
                val_labels_accumulated.extend(val_labels)
                num_val_images = len(val_images_accumulated)


                # 編譯模型
                model.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # 訓練模型
                model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_data=(np.array(val_images_accumulated), np.array(val_labels_accumulated)), verbose=0)

                # 保存訓練好的模型
                model.save(model_path)

                # 清空資料
                images = []
                labels = []

                

# 確保最後一批資料也被訓練到
if len(images) > 0:
    images_np = np.array(images)
    labels_np = np.array(labels)
    labels_one_hot = to_categorical(labels_np, num_classes=10)
    train_images, val_images, train_labels, val_labels = train_test_split(images_np, labels_one_hot, test_size=0.3, random_state=42)

    # 累積驗證集的資料
    val_images_accumulated.extend(val_images)
    val_labels_accumulated.extend(val_labels)

    model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_data=(np.array(val_images_accumulated), np.array(val_labels_accumulated)), verbose=0)

print('模型訓練完成')

num_val_images = len(val_images_accumulated)
print("驗證集中的圖片數量:", num_val_images)

# 將 val_images_accumulated 中的圖像形狀從 (900, 64, 64) 轉換為 (900, 64, 64, 1)
val_images_accumulated1 = np.expand_dims(val_images_accumulated, axis=-1)

# 使用模型預測
val_pred = model.predict(val_images_accumulated1, verbose=0)
val_pred_labels = np.argmax(val_pred, axis=1)
val_true_labels = np.argmax(val_labels_accumulated, axis=1)
val_conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

# 驗證集Top-1
val_top1_acc = accuracy_score(val_true_labels, val_pred_labels)
val_top1_num = np.sum(val_true_labels == val_pred_labels)
val_top1_denom = len(val_true_labels)

# 驗證集Top-3
val_top3_acc = np.mean([1 if true_label in pred_labels.argsort()[-3:] else 0 for true_label, pred_labels in zip(val_true_labels, val_pred)])
val_top3_num = np.sum([1 if true_label in pred_labels.argsort()[-3:] else 0 for true_label, pred_labels in zip(val_true_labels, val_pred)])
val_top3_denom = len(val_true_labels)

# 打印驗證集指標
print("1.")
print("\n(1)The confusion matrix:")
print(val_conf_matrix)
print("\n(2)Top-1 Accuracy:", val_top1_acc)
print("Top-1 分子:", val_top1_num)
print("Top-1 分母:", val_top1
      _denom)
print("\n(3)Top-3 Accuracy:", val_top3_acc)
print("Top-3 分子:", val_top3_num)
print("Top-3 分母:", val_top3_denom)

檔案 'saved_model.h5' 不存在，無需刪除。
模型訓練完成
驗證集中的圖片數量: 29219
1.

(1)The confusion matrix:
[[3150    7   12    2    3   10   36    0    7    7]
 [   2 3180    6    6   19    6    3    4    0    4]
 [  22   37 2580  125    5   12    3   28   12    6]
 [   1    0   11 2892    0   14    0    6    7    7]
 [   1   14    2    3 2686    5    7    2    2   48]
 [   1    2    1   61    2 2592   12    2    5    4]
 [  20   17    0    0   12   19 2750    0    4    0]
 [   3   20   11   42   18    1    0 2869    1   82]
 [  46   12   12   59   21   76   46    2 2454   61]
 [  18    5    1   37   61   19    2   19    9 2706]]

(2)Top-1 Accuracy: 0.9534549437010165
Top-1 分子: 27859
Top-1 分母: 29219

(3)Top-3 Accuracy: 0.9946952325541599
Top-3 分子: 29064
Top-3 分母: 29219


In [28]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# 刪除檔案
model_path = 'test_results.csv'
if os.path.exists(model_path):
    os.remove(model_path)
    print(f"檔案 '{model_path}' 已成功刪除。")
else:
    print(f"檔案 '{model_path}' 不存在，無需刪除。")
    
model = load_model('/kaggle/working/saved_model.h5')

# Path to the folder containing test images
test_folder_path = '/kaggle/input/advancedml-hw2/hw2/HW2_MNIST_test'

# Initialize lists to store image paths and their corresponding predictions
test_image_paths = []
predictions = []

# Loop through each file in the test folder
for filename in os.listdir(test_folder_path):
    # Check if the file is an image
    if filename.endswith('.png'):
        # Read the image
        image_path = os.path.join(test_folder_path, filename)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64))
        if img is None:
            print(f"Error reading image: {image_path}")
            continue
        digit_img = img.astype(np.float32) / 255.0

        # Predict the label using the trained model
        pred = model.predict(np.expand_dims(digit_img, axis=0), verbose=0)  # Use np.expand_dims to add batch dimension
        predicted_label = np.argmax(pred)

        # Store the image path and predicted label
        test_image_paths.append(filename)
        predictions.append(predicted_label)

# Create a DataFrame to store the results
results_df = pd.DataFrame({'image': test_image_paths, 'class': predictions})

# Write the DataFrame to a CSV file
results_df.to_csv('HW2_prob1.csv', index=False)
print('完成預測檔 HW2_prob1.csv')

檔案 'test_results.csv' 不存在，無需刪除。
完成預測檔 HW2_prob1.csv
